1. Data Processing

In [1]:
# 1. ИМПОРТ БИБЛИОТЕК
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import os
import re


import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
from plotly.offline import plot
from scipy import stats

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, QuantileTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [2]:
# Настройки отображения
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [3]:
# Загрузка данных
# sales = pd.read_excel('Sales.xlsx', dtype={'OrderDate': str, 'ShipDate': str})
sales = pd.read_excel('Sales.xlsx')
customer = pd.read_excel('Customer.xlsx')
product = pd.read_excel('Product.xlsx')
territories = pd.read_excel('Territories.xlsx')

In [4]:
# Консолидация данных
merged_data = sales.merge(customer, on='CustomerKey', how='left') \
                  .merge(product, on='ProductKey', how='left') \
                  .merge(territories, left_on='SalesTerritoryKey', right_on='SalesTerritoryKey', how='left')

# Проверка
print("Размер объединённого датасета:", merged_data.shape)
print("\nПервые 5 строк:")
print(merged_data.head())  

Размер объединённого датасета: (58189, 50)

Первые 5 строк:
   ProductKey  OrderDate   ShipDate  CustomerKey  PromotionKey  \
0         310 2014-01-01 2014-01-08        21768             1   
1         346 2014-01-01 2014-01-08        28389             1   
2         346 2014-01-01 2014-01-08        25863             1   
3         336 2014-01-01 2014-01-08        14501             1   
4         346 2014-01-01 2014-01-08        11003             1   

   SalesTerritoryKey SalesOrderNumber  SalesOrderLineNumber  OrderQuantity  \
0                  6          SO43697                     1              2   
1                  7          SO43698                     1              2   
2                  1          SO43699                     1              2   
3                  4          SO43700                     1              2   
4                  9          SO43701                     1              2   

   UnitPrice  TotalProductCost  SalesAmount    TaxAmt FirstName  LastName 

In [5]:
'''5. Печать информации о очищенных данных'''
print("\nИнформация об очищенных данных:")
print(merged_data.info())


Информация об очищенных данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58189 entries, 0 to 58188
Data columns (total 50 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ProductKey            58189 non-null  int64         
 1   OrderDate             58189 non-null  datetime64[ns]
 2   ShipDate              58189 non-null  datetime64[ns]
 3   CustomerKey           58189 non-null  int64         
 4   PromotionKey          58189 non-null  int64         
 5   SalesTerritoryKey     58189 non-null  int64         
 6   SalesOrderNumber      58189 non-null  object        
 7   SalesOrderLineNumber  58189 non-null  int64         
 8   OrderQuantity         58189 non-null  int64         
 9   UnitPrice             58189 non-null  float64       
 10  TotalProductCost      58189 non-null  float64       
 11  SalesAmount           58189 non-null  float64       
 12  TaxAmt                58189 non-null  flo

In [ ]:
'''6. Сохранение объединенных данных'''
merged_data.to_csv('6_merged_data.csv', index=False)

print("Очистка данных завершена!")

Очистка данных завершена!


In [23]:
# Дополнительные признаки
merged_data['Profit'] = merged_data['SalesAmount'] - merged_data['TotalProductCost']
merged_data['OrderYear'] = merged_data['OrderDate'].dt.year
merged_data['OrderMonth'] = merged_data['OrderDate'].dt.month

print("Данные успешно загружены и подготовлены.")
print(f"Размер объединённого датасета: {merged_data.shape}")
print("\nПервые 5 строк:")
print(merged_data.head())

AttributeError: Can only use .dt accessor with datetimelike values

In [22]:
# Удаление столбцов
columns_to_drop = [
    'Photo',
    'ProductDescription',
    'NumberChildrenAtHome',
    'AddressLine1',
    'RegionImage',
    'RegionInfo',
    'CustomerStateCode',
    
    'SalesOrderLineNumber',
    'StandardCost',  # равен TotalProductCost
    'ListPrice',     # равен SalesAmount
    'PromotionKey',
    'CustomerCountry', # равен Country
    'Group'
]

merged_data = merged_data.drop(columns=columns_to_drop, errors='ignore')



In [ ]:
'''5. Печать информации о очищенных данных'''
print("\nИнформация об очищенных данных:")
print(merged_data.info())

In [ ]:
'''1. Обработка пропусков'''
'''Печатаем количество пропущенных значений в каждом столбце'''
print("\nКоличество пропущенных значений:")
print(merged_data.isnull().sum())
print(merged_data.info())

In [ ]:
# Удаляем строки с любыми пропусками
merged_data.dropna(inplace=True)
merged_data.describe()

In [12]:
# Преобразование числовых данных
merged_data['UnitPrice'] = pd.to_numeric(merged_data['UnitPrice'], errors='coerce')
merged_data['TotalProductCost'] = pd.to_numeric(merged_data['TotalProductCost'], errors='coerce')
merged_data['SalesAmount'] = pd.to_numeric(merged_data['SalesAmount'], errors='coerce')
merged_data['TaxAmt'] = pd.to_numeric(merged_data['TaxAmt'], errors='coerce')
merged_data['YearlyIncome'] = pd.to_numeric(merged_data['YearlyIncome'], errors='coerce')


In [ ]:
'''3. Приведение типов'''
'''Приведение столбца 'OrderDate' к типу datetime'''
# merged_data['OrderDate'] = pd.to_datetime(merged_data['OrderDate'], errors='coerce')
# merged_data['ShipDate'] = pd.to_datetime(merged_data['ShipDate'], errors='coerce')
# merged_data['BirthDate'] = pd.to_datetime(merged_data['BirthDate'], errors='coerce')

date_columns = [col for col in merged_data.columns if 'Date' in col]
for col in date_columns:
    merged_data[col] = pd.to_datetime(merged_data[col], errors='coerce').dt.date

In [ ]:
'''5. Печать информации о очищенных данных'''
print("\nИнформация об очищенных данных:")
print(merged_data.info())

In [ ]:
'''6. Сохранение очищенных данных'''
merged_data.to_csv('cleaned_data.csv', index=False)

print("Очистка данных завершена!")

In [ ]:
# 🧹 ОЧИСТКА И ПРЕОБРАЗОВАНИЕ ДАННЫХ
print("🧹 ОЧИСТКА ДАННЫХ")
print("=" * 50)
df = data
# Сохраняем исходный размер
initial_count = len(df)
print(f"📊 Исходное количество строк: {initial_count}")

# Удаление дубликатов
df = df.drop_duplicates()
duplicates_removed = initial_count - len(df)
print(f"📊 Удалено дубликатов: {duplicates_removed}")

# Анализ пропущенных значений
print("\n📊 АНАЛИЗ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ:")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_info = pd.DataFrame({
    'Количество': missing_data,
    'Процент': missing_percent
}).sort_values('Количество', ascending=False)

# Показываем только колонки с пропусками
missing_columns = missing_info[missing_info['Количество'] > 0]
if len(missing_columns) > 0:
    print("Колонки с пропущенными значениями:")
    display(missing_columns)
else:
    print("✅ Пропущенных значений не обнаружено")

# Заполнение пропусков для ключевых колонок
print("\n🔧 ЗАПОЛНЕНИЕ ПРОПУСКОВ:")
if 'Color' in df.columns:
    df['Color'] = df['Color'].fillna('Not Specified')
    print("✅ Color: заполнены пропуски значением 'Not Specified'")

if 'SubCategory' in df.columns:
    df['SubCategory'] = df['SubCategory'].fillna('Unknown')
    df['Category'] = df['Category'].fillna('Unknown')
    print("✅ Category/SubCategory: заполнены пропуски значением 'Unknown'")

# Преобразование типов данных
print("\n🔄 ПРЕОБРАЗОВАНИЕ ТИПОВ ДАННЫХ:")

date_columns = ['OrderDate', 'ShipDate', 'BirthDate', 'DateFirstPurchase', 'StartDate']
for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"✅ {col}: преобразована в datetime")

# Преобразование числовых колонок
numeric_columns = ['YearlyIncome', 'TotalChildren', 'NumberChildrenAtHome', 
                  'NumberCarsOwned', 'StandardCost', 'ListPrice', 'DaysToManufacture',
                  'OrderQuantity', 'UnitPrice', 'TotalProductCost', 'SalesAmount', 'TaxAmt']

for col in numeric_columns:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        print(f"✅ {col}: преобразована в числовой формат")

# Создание новых признаков
print("\n🎯 СОЗДАНИЕ НОВЫХ ПРИЗНАКОВ:")

if 'OrderDate' in df.columns:
    df['OrderYear'] = df['OrderDate'].dt.year
    df['OrderMonth'] = df['OrderDate'].dt.month
    df['OrderQuarter'] = df['OrderDate'].dt.quarter
    df['OrderDayOfWeek'] = df['OrderDate'].dt.day_name()
    print("✅ Созданы временные признаки из OrderDate")
    
    # Расчет времени доставки
    if 'ShipDate' in df.columns:
        df['DeliveryDays'] = (df['ShipDate'] - df['OrderDate']).dt.days
        print("✅ Создан признак DeliveryDays")

# Создание демографических сегментов
if 'YearlyIncome' in df.columns:
    df['IncomeSegment'] = pd.cut(df['YearlyIncome'], 
                               bins=[0, 50000, 80000, 100000, float('inf')],
                               labels=['Low', 'Medium', 'High', 'Very High'])
    print("✅ Создан IncomeSegment")

# Расчет возраста клиента
if 'BirthDate' in df.columns:
    df['Age'] = (datetime.now() - df['BirthDate']).dt.days // 365
    print("✅ Создан признак Age")

# Расчет прибыльности
if 'SalesAmount' in df.columns and 'TotalProductCost' in df.columns:
    df['Profit'] = df['SalesAmount'] - df['TotalProductCost']
    df['ProfitMargin'] = (df['Profit'] / df['SalesAmount']) * 100
    print("✅ Созданы Profit и ProfitMargin")

# Расчет общей суммы, если нет
if 'UnitPrice' in df.columns and 'OrderQuantity' in df.columns and 'TotalAmount' not in df.columns:
    df['TotalAmount'] = df['UnitPrice'] * df['OrderQuantity']
    print("✅ Создан TotalAmount")

print(f"\n📊 РЕЗУЛЬТАТ ОЧИСТКИ:")
print(f"   • Исходный размер: {initial_count} строк")
print(f"   • Финальный размер: {len(df)} строк") 
print(f"   • Удалено дубликатов: {duplicates_removed}")
print(f"   • Количество колонок: {len(df.columns)}")

print("\n👀 ПРЕВЬЮ ОЧИЩЕННЫХ ДАННЫХ:")
display(df.head(3))

print("\n📋 ИНФОРМАЦИЯ О ДАННЫХ ПОСЛЕ ОЧИСТКИ:")
print(df.info())

print("\n✅ ДАННЫЕ ОЧИЩЕНЫ И ПРЕОБРАЗОВАНЫ!")
'''6. Сохранение очищенных данных'''
data_cleaned.to_csv('cleaned_data.csv', index=False)

print("Очистка данных завершена!")

In [ ]:
'''6. Сохранение объединенных данных данных'''
merged_data.to_csv('merged_data.csv', index=False)

print("Очистка данных завершена!")

In [ ]:
'''6. Сохранение объединенных данных данных'''
merged_data.to_csv('merged_data.csv', index=False)

print("Очистка данных завершена!")

In [ ]:
'''1. Загрузка данных'''
data = pd.read_csv('merged_sales_data.csv')

In [ ]:
merged_data.to_excel('output_file.xlsx', index=False)

In [ ]:
'''2. Удаление дубликатов'''
duplicates_count = data_cleaned.duplicated().sum()
print(f"\nКоличество дубликатов: {duplicates_count}")
data_cleaned = data_cleaned.drop_duplicates()

In [ ]:
'''Просмотр первых 5 строк'''
print("Исходные данные:")
print(data.head())

In [ ]:
'''1. Удаление строк/столбцов'''
df_dropped_rows = df.dropna()
df_dropped_cols = df.dropna(axis=1)

In [ ]:
'''Удаление строк с пропущенными значениями'''
data_cleaned = data.dropna()

In [ ]:
'''2. Удаление дубликатов'''
duplicates_count = data_cleaned.duplicated().sum()
print(f"\nКоличество дубликатов: {duplicates_count}")
data_cleaned = data_cleaned.drop_duplicates()

In [ ]:
'''2. Заполнение средними значениями'''
mean_imputer = SimpleImputer(strategy='mean')
df_mean_filled = pd.DataFrame(mean_imputer.fit_transform(df), columns=df.columns)

нормализация и преобразование признаков

In [ ]:
'''4. Удаление выбросов'''
'''Пример: удаление выбросов в числовом столбце 'value_column'''
Q1 = data_cleaned['Price'].quantile(0.25)
Q3 = data_cleaned['Price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

data_cleaned = data_cleaned[(data_cleaned['Price'] >= lower_bound) &
                             (data_cleaned['Price'] <= upper_bound)]

In [ ]:
# Загрузка данных
df = pd.read_csv('merged_data.csv')  

# Выбираем только числовые столбцы
numeric_df = df.select_dtypes(include=[np.number])

# Проверка: какие столбцы числовые
print("Числовые столбцы:")
print(numeric_df.columns.tolist())

# Если числовые столбцы есть, применяем нормализацию
if numeric_df.empty:
    print("Нет числовых столбцов для нормализации.")
else:
    # 1. Z-score Normalization
    z_score_scaler = StandardScaler()
    z_score_normalized = z_score_scaler.fit_transform(numeric_df)

    # 2. Min-Max Normalization
    min_max_scaler = MinMaxScaler()
    min_max_normalized = min_max_scaler.fit_transform(numeric_df)

    # 3. Robust Scaler
    robust_scaler = RobustScaler()
    robust_normalized = robust_scaler.fit_transform(numeric_df)

    # 4. MaxAbs Scaler
    max_abs_scaler = MaxAbsScaler()
    max_abs_normalized = max_abs_scaler.fit_transform(numeric_df)

    # 5. Log Transformation (только для положительных значений)
    positive_data = numeric_df[numeric_df > 0].fillna(0)
    log_transformer = FunctionTransformer(np.log1p, validate=True)
    log_transformed = log_transformer.fit_transform(positive_data)

    # 6. Quantile Transformation
    quantile_transformer = QuantileTransformer(output_distribution='uniform')
    quantile_normalized = quantile_transformer.fit_transform(numeric_df)

    # 7. Decimal Scaling
    def decimal_scaling(data):
        j = np.ceil(np.log10(np.abs(data).max(axis=0) + 1))
        return data / (10 ** j)

    decimal_scaled = decimal_scaling(numeric_df.values)

    # 8. Softmax (только для положительных значений)
    def softmax(x):
        exp_x = np.exp(x - np.max(x, axis=0))
        return exp_x / exp_x.sum(axis=0)

    softmax_normalized = softmax(numeric_df.values)

    '''Вывод результатов'''
    print("\nZ-score Normalized (первые 5 строк):\n", z_score_normalized[:5])
    print("\nMin-Max Normalized (первые 5 строки):\n", min_max_normalized[:5])
    print("\nRobust Normalized (первые 5 строки):\n", robust_normalized[:5])
    print("\nMaxAbs Normalized (первые 5 строки):\n", max_abs_normalized[:5])
    print("\nLog Transformed (первые 5 строки):\n", log_transformed[:5])
    print("\nQuantile Normalized (первые 5 строки):\n", quantile_normalized[:5])
    print("\nDecimal Scaled (первые 5 строки):\n", decimal_scaled[:5])
    print("\nSoftmax Normalized (первые 5 строки):\n", softmax_normalized[:5])

In [ ]:
df = pd.read_csv('merged_data.csv')

In [ ]:
'''1. Тест Граббса (Grubbs' Test)'''
def grubbs_test(data):
    N = len(data)
    mean = np.mean(data)
    std_dev = np.std(data)
    G = max(abs(data - mean)) / std_dev
    critical_value = stats.t.ppf(1 - 0.05 / (2 * N), N - 2)  # для 5% уровня значимости
    return G, G > critical_value

grubbs_result = grubbs_test(data)
print("Grubbs' Test: G =", grubbs_result[0], "Is outlier?", grubbs_result[1])

In [ ]:
'''2. Тест Диксона (Dixon's Q Test)'''
def dixon_test(data):
    data_sorted = np.sort(data)
    Q_low = (data_sorted[1] - data_sorted[0]) / (data_sorted[-1] - data_sorted[0])
    Q_high = (data_sorted[-1] - data_sorted[-2]) / (data_sorted[-1] - data_sorted[0])
    critical_value = 0.5  # Порог для Q (исходя из таблиц для n = 10)
    return Q_low > critical_value or Q_high > critical_value

dixon_result = dixon_test(data)
print("Dixon's Q Test: Is outlier?", dixon_result)

In [ ]:
'''4. Тест Розенбаума (Rosner's Test)'''
def rosner_test(data, max_outliers=1):
    N = len(data)
    data_sorted = np.sort(data)
    for i in range(max_outliers):
        mean = np.mean(data_sorted)
        std_dev = np.std(data_sorted)
        threshold = mean + (i + 1) * std_dev
        if data_sorted[-(i + 1)] > threshold:
            return True
    return False

rosner_result = rosner_test(data)
print("Rosner's Test: Is outlier?", rosner_result)

In [ ]:
'''5. Тест Тьо-Гентера (Tietjen-Moore Test)'''
def tietjen_moore_test(data):
    N = len(data)
    mean = np.mean(data)
    std_dev = np.std(data)
    z_scores = (data - mean) / std_dev
    return np.any(np.abs(z_scores) > 3)  # Порог Z-значения

tietjen_moore_result = tietjen_moore_test(data)
print("Tietjen-Moore Test: Is outlier?", tietjen_moore_result)

In [ ]:
'''6. Тест Стьюдента (Student's t-test)'''
t_statistic, p_value = stats.ttest_1samp(data, 0)
print("Student's t-test: t-statistic =", t_statistic, ", p-value =", p_value)

In [ ]:
'''7. Тест Спирмена (Spearman's Rank Correlation Coefficient)'''
spearman_corr, spearman_p = stats.spearmanr(data, np.arange(len(data)))
print("Spearman's Test: Correlation =", spearman_corr, ", p-value =", spearman_p)

In [ ]:
'''8. Тест Манна-Уитни (Mann-Whitney U Test)'''
# Для примера создадим два набора данных
data1 = np.array([1, 2, 3, 4, 5])
data2 = np.array([5, 6, 7, 8, 9])
mann_whitney_result = stats.mannwhitneyu(data1, data2)
print("Mann-Whitney U Test: U-statistic =", mann_whitney_result.statistic, ", p-value =", mann_whitney_result.pvalue)

In [ ]:
'''9. Тест Шапиро-Уилка (Shapiro-Wilk Test)'''
shapiro_stat, shapiro_p = stats.shapiro(data)
print("Shapiro-Wilk Test: W-statistic =", shapiro_stat, ", p-value =", shapiro_p)

In [ ]:
'''10. Тест Колмогорова-Смирнова (Kolmogorov-Smirnov Test)'''
ks_stat, ks_p = stats.kstest(data, 'norm', args=(np.mean(data), np.std(data)))
print("Kolmogorov-Smirnov Test: D-statistic =", ks_stat, ", p-value =", ks_p)

In [ ]:
'''3. Критерий Шовене (Chauvenet's Criterion)'''
def chauvenet_test(data):
    N = len(data)
    mean = np.mean(data)
    std_dev = np.std(data)
    d = abs(data - mean) / std_dev
    p_values = 1 / (2 * N * np.exp(0.5 * (d**2)))
    return np.any(p_values < 0.5)

chauvenet_result = chauvenet_test(data)
print("Chauvenet's Criterion: Is outlier?", chauvenet_result)

In [ ]:
# 🧹 ОЧИСТКА И ПРЕОБРАЗОВАНИЕ ДАННЫХ
print("🧹 ОЧИСТКА ДАННЫХ")
print("=" * 50)
df = data
# Сохраняем исходный размер
initial_count = len(df)
print(f"📊 Исходное количество строк: {initial_count}")

# Удаление дубликатов
df = df.drop_duplicates()
duplicates_removed = initial_count - len(df)
print(f"📊 Удалено дубликатов: {duplicates_removed}")

# Анализ пропущенных значений
print("\n📊 АНАЛИЗ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ:")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_info = pd.DataFrame({
    'Количество': missing_data,
    'Процент': missing_percent
}).sort_values('Количество', ascending=False)

# Показываем только колонки с пропусками
missing_columns = missing_info[missing_info['Количество'] > 0]
if len(missing_columns) > 0:
    print("Колонки с пропущенными значениями:")
    display(missing_columns)
else:
    print("✅ Пропущенных значений не обнаружено")

# Заполнение пропусков для ключевых колонок
print("\n🔧 ЗАПОЛНЕНИЕ ПРОПУСКОВ:")
if 'Color' in df.columns:
    df['Color'] = df['Color'].fillna('Not Specified')
    print("✅ Color: заполнены пропуски значением 'Not Specified'")

if 'SubCategory' in df.columns:
    df['SubCategory'] = df['SubCategory'].fillna('Unknown')
    df['Category'] = df['Category'].fillna('Unknown')
    print("✅ Category/SubCategory: заполнены пропуски значением 'Unknown'")

# Преобразование типов данных
print("\n🔄 ПРЕОБРАЗОВАНИЕ ТИПОВ ДАННЫХ:")

date_columns = ['OrderDate', 'ShipDate', 'BirthDate', 'DateFirstPurchase', 'StartDate']
for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"✅ {col}: преобразована в datetime")

# Преобразование числовых колонок
numeric_columns = ['YearlyIncome', 'TotalChildren', 'NumberChildrenAtHome', 
                  'NumberCarsOwned', 'StandardCost', 'ListPrice', 'DaysToManufacture',
                  'OrderQuantity', 'UnitPrice', 'TotalProductCost', 'SalesAmount', 'TaxAmt']

for col in numeric_columns:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        print(f"✅ {col}: преобразована в числовой формат")

# Создание новых признаков
print("\n🎯 СОЗДАНИЕ НОВЫХ ПРИЗНАКОВ:")

if 'OrderDate' in df.columns:
    df['OrderYear'] = df['OrderDate'].dt.year
    df['OrderMonth'] = df['OrderDate'].dt.month
    df['OrderQuarter'] = df['OrderDate'].dt.quarter
    df['OrderDayOfWeek'] = df['OrderDate'].dt.day_name()
    print("✅ Созданы временные признаки из OrderDate")
    
    # Расчет времени доставки
    if 'ShipDate' in df.columns:
        df['DeliveryDays'] = (df['ShipDate'] - df['OrderDate']).dt.days
        print("✅ Создан признак DeliveryDays")

# Создание демографических сегментов
if 'YearlyIncome' in df.columns:
    df['IncomeSegment'] = pd.cut(df['YearlyIncome'], 
                               bins=[0, 50000, 80000, 100000, float('inf')],
                               labels=['Low', 'Medium', 'High', 'Very High'])
    print("✅ Создан IncomeSegment")

# Расчет возраста клиента
if 'BirthDate' in df.columns:
    df['Age'] = (datetime.now() - df['BirthDate']).dt.days // 365
    print("✅ Создан признак Age")

# Расчет прибыльности
if 'SalesAmount' in df.columns and 'TotalProductCost' in df.columns:
    df['Profit'] = df['SalesAmount'] - df['TotalProductCost']
    df['ProfitMargin'] = (df['Profit'] / df['SalesAmount']) * 100
    print("✅ Созданы Profit и ProfitMargin")

# Расчет общей суммы, если нет
if 'UnitPrice' in df.columns and 'OrderQuantity' in df.columns and 'TotalAmount' not in df.columns:
    df['TotalAmount'] = df['UnitPrice'] * df['OrderQuantity']
    print("✅ Создан TotalAmount")

print(f"\n📊 РЕЗУЛЬТАТ ОЧИСТКИ:")
print(f"   • Исходный размер: {initial_count} строк")
print(f"   • Финальный размер: {len(df)} строк") 
print(f"   • Удалено дубликатов: {duplicates_removed}")
print(f"   • Количество колонок: {len(df.columns)}")

print("\n👀 ПРЕВЬЮ ОЧИЩЕННЫХ ДАННЫХ:")
display(df.head(3))

print("\n📋 ИНФОРМАЦИЯ О ДАННЫХ ПОСЛЕ ОЧИСТКИ:")
print(df.info())

print("\n✅ ДАННЫЕ ОЧИЩЕНЫ И ПРЕОБРАЗОВАНЫ!")
'''6. Сохранение очищенных данных'''
data_cleaned.to_csv('cleaned_data.csv', index=False)

print("Очистка данных завершена!")

In [ ]:
# 🔍 ИССЛЕДОВАТЕЛЬСКИЙ АНАЛИЗ ДАННЫХ
print("🔍 ИССЛЕДОВАТЕЛЬСКИЙ АНАЛИЗ ДАННЫХ")
print("=" * 50)

# Базовая статистика числовых колонок
numeric_cols = df.select_dtypes(include=[np.number]).columns
print("📈 ОСНОВНАЯ СТАТИСТИКА (числовые колонки):")
print(f"📊 Найдено числовых колонок: {len(numeric_cols)}")

if len(numeric_cols) > 0:
    display(df[numeric_cols].describe())
else:
    print("❌ Числовые колонки не найдены")

# Анализ категориальных переменных
categorical_cols = df.select_dtypes(include=['object']).columns
print(f"\n📊 КАТЕГОРИАЛЬНЫЕ ПЕРЕМЕННЫЕ ({len(categorical_cols)}):")

if len(categorical_cols) > 0:
    for i, col in enumerate(categorical_cols[:8], 1):  # Показываем первые 8
        unique_count = df[col].nunique()
        print(f"   {i}. {col}: {unique_count} уникальных значений")
        if unique_count <= 10:
            value_counts = df[col].value_counts()
            print(f"      Распределение: {dict(value_counts)}")
        else:
            print(f"      Первые 5 значений: {list(df[col].unique()[:5])}")
    
    if len(categorical_cols) > 8:
        print(f"   ... и еще {len(categorical_cols) - 8} категориальных колонок")
else:
    print("❌ Категориальные колонки не найдены")

# Анализ временных колонок
date_cols = df.select_dtypes(include=['datetime64']).columns
print(f"\n🕐 ВРЕМЕННЫЕ ПЕРЕМЕННЫЕ ({len(date_cols)}):")

if len(date_cols) > 0:
    for i, col in enumerate(date_cols, 1):
        date_range = f"{df[col].min().date()} - {df[col].max().date()}"
        print(f"   {i}. {col}: {date_range}")
else:
    print("❌ Временные колонки не найдены")

# Общая информация о данных
print(f"\n📋 ОБЩАЯ ИНФОРМАЦИЯ О ДАННЫХ:")
print(f"   • Размер данных: {df.shape[0]} строк, {df.shape[1]} колонок")
print(f"   • Типы данных:")
print(f"      - Числовые: {len(numeric_cols)}")
print(f"      - Категориальные: {len(categorical_cols)}")
print(f"      - Временные: {len(date_cols)}")
print(f"      - Другие: {df.shape[1] - len(numeric_cols) - len(categorical_cols) - len(date_cols)}")

# Пропущенные значения
missing_total = df.isnull().sum().sum()
missing_percent = (missing_total / (df.shape[0] * df.shape[1])) * 100
print(f"   • Пропущенные значения: {missing_total} ({missing_percent:.2f}%)")

print("\n👀 ПЕРВЫЕ 3 СТРОКИ ДАННЫХ:")
display(df.head(3))

print("✅ ИССЛЕДОВАТЕЛЬСКИЙ АНАЛИЗ ЗАВЕРШЕН!")